In [ ]:
!pip install openai
!pip install gradio
!pip install langchain
!pip install python-dotenv

In [11]:
# Import necessary libraries
import os
import openai
import pandas as pd
from gradio import Interface
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from chatbot.embeddings import generate_embeddings, search_embedding, load_embeddings_data
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [12]:
# Initialize OpenAI model
llm_openai = OpenAI(model_name = "text-davinci-003", openai_api_key=openai.api_key)

In [27]:
# Initialize ConversationBufferMemory and ConversationChain
memory = ConversationBufferMemory()
chatbot_chain = ConversationChain(llm=llm_openai, memory=memory, verbose=True)

In [15]:
# Define sending prompt function
def send_prompt(prompt, engine="text-davinci-003", temp=0.5, max_tokens=100, top_p=1, frequency_penalty=0, presence_penalty=0):
    response = openai.Completion.create(
        engine=engine,
        prompt=prompt,
        temperature=temp,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty
    )
    return response['choices'][0]['text']

In [20]:
# Define search function using embeddings
def search_embedding(query, data, n_results=5):
    query_embed = generate_embeddings(query, engine="text-embedding-ada-002")
    data["Similarity"] = data['Embedding'].apply(lambda x: search_embedding(x, query_embed))
    data = data.sort_values("Similarity", ascending=False)
    return data.iloc[:n_results][["text", "Similarity"]]

In [30]:
# Define chat function
def chat(input_text):
    response = chatbot_chain.predict(input = input_text)
    return response

In [31]:
# Define main function
def main():
    # Load embeddings
    embeddings_data = load_embeddings_data('data/processed/embeddings.csv')
    
    # Gradio interface
    iface = Interface(
        fn=chat, 
        inputs="text", 
        outputs="text", 
        examples=[
            ["What's the weather like today?"],
            ["Tell me a joke."],
            ["Generate a poem about the sea."],
        ]
    )
    iface.launch()


if __name__ == "__main__":
    main()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Como me llamo?
AI:  Tu nombre es [Nombre del usuario]. ¿Hay algo más en lo que pueda ayudarte?
Human: hola
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Como me llamo?
AI:  Tu nombre es [Nombre del usuario]. ¿Hay algo más en lo que pueda ayudarte?
Human: hola
AI:  Hola, ¿en qué puedo ayudarte?
Human: Como me llamo?
AI:

> Finished chain.
